# Efficient Quantum Kernel Evaluation Demo
In this notebook we create a 4x4 symmetric fidelity quantum kernel where every entry is calculated by a 2 qubits circuit. We run our code both on a simulator and on a real QPU to demonstrate our Efficient Quantum Kernel Evaluation implementation.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Session, SamplerV2
import numpy as np
from qiskit.circuit.library import ZZFeatureMap
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager   
from qiskit_machine_learning.state_fidelities import ComputeUncompute    


In [7]:
#we import our efficient implemetation of the FidelityQuantumKernel which is compatible with the qiskit-machine-learning implementation

from fidelity_quantum_kernel_efficient import FidelityQuantumKernel 

We now create a dataset which will be used to to evaluate the kernel

In [4]:
num_features = 2         #this is the dimensionality of every point in the dataset 
#we map every feature to a qubit in the fidelity quantum circuit used to compute each kernel entry. 
#Therefore, num_features is also the number of qubits used by the fidelity circuit.
number_of_points = 4

#create data
np.random.seed(seed=123)
X = np.random.uniform(-np.pi/2, np.pi/2, size = (number_of_points,num_features))

We coonect to the qiskit runtime service to both a simulator and a real QPU. We then create a feature map which will be used to compute the fidelity and create an instance 

In [9]:
service = QiskitRuntimeService()

#connection to a real backend
backend_QPU = service.least_busy(simulator=False, operational=True)  # to select the least busy quantum machine available
num_qubits_backend_QPU = backend_QPU.num_qubits
session_QPU = Session(backend=backend_QPU)
pass_manager_QPU = generate_preset_pass_manager(optimization_level=1, backend=backend_QPU)
sampler_QPU = SamplerV2(mode=session_QPU)

#connection to a simulator
num_qubits_backend_sim = 20
backend_sim = GenericBackendV2(num_qubits=num_qubits_backend_sim, seed = 1)                         
session_sim = Session(backend=backend_sim)
pass_manager_sim = generate_preset_pass_manager(optimization_level=1, backend=backend_sim)
sampler_sim = SamplerV2(mode=session_sim)


#define a feature map 
feature_map = ZZFeatureMap(feature_dimension=num_features, reps=1, entanglement='linear')

In order to evaluate the kernel on the data one simply needs to call the .evaluate method (identical to the qiskit-machine-learning implementation) or the .evaluate_efficient method, which will parallelize the fidelity circuits runs when running on a real QPU. 

In [ ]:
# CASE 1, efficient implementation on a simulator

fidelity_efficient_sim = ComputeUncompute(sampler=sampler_sim)
qkernel_efficient_sim = FidelityQuantumKernel(feature_map = feature_map, fidelity = fidelity_efficient_sim)
kernel_matrix_efficient_sim = qkernel_efficient_sim.evaluate_efficient(X, backend_sim, pass_manager = pass_manager_sim, shots = 1024)

print(kernel_matrix_efficient_sim)


c:\Users\melch\.conda\envs\qpu_demo\Lib\site-packages\qiskit\providers\fake_provider\generic_backend_v2.py:506: RuntimeWarning: Aer not found using BasicSimulator and no noise
  warnings.warn("Aer not found using BasicSimulator and no noise", RuntimeWarning)


[[1.         0.29101562 0.0390625  0.30175781]
 [0.29101562 1.         0.04296875 0.09472656]
 [0.0390625  0.04296875 1.         0.26074219]
 [0.30175781 0.09472656 0.26074219 1.        ]]


In [ ]:
# CASE 2, efficient implementation on a QPU

fidelity_efficient_QPU = ComputeUncompute(sampler=sampler_QPU)
qkernel_efficient_QPU = FidelityQuantumKernel(feature_map = feature_map, fidelity = fidelity_efficient_QPU)
kernel_matrix_efficient_QPU = qkernel_efficient_QPU.evaluate_efficient(X, backend_QPU, pass_manager = pass_manager_QPU, shots = 1024)

print(kernel_matrix_efficient_QPU)  

#NOTE: this run used 1 second and 1 job on  ibm_sherbrooke

[[1.         0.31640625 0.02929688 0.23828125]
 [0.31640625 1.         0.11621094 0.078125  ]
 [0.02929688 0.11621094 1.         0.31152344]
 [0.23828125 0.078125   0.31152344 1.        ]]


In [11]:
# CASE 3, standard implementation on a simulator


fidelity_sim = ComputeUncompute(sampler=sampler_sim,  pass_manager=pass_manager_sim)
qkernel_sim = FidelityQuantumKernel(feature_map = feature_map, fidelity = fidelity_sim)
kernel_matrix_sim = qkernel_sim.evaluate(X)

print(kernel_matrix_sim)

c:\Users\melch\.conda\envs\qpu_demo\Lib\site-packages\qiskit\providers\fake_provider\generic_backend_v2.py:506: RuntimeWarning: Aer not found using BasicSimulator and no noise
  warnings.warn("Aer not found using BasicSimulator and no noise", RuntimeWarning)


[[1.         0.30957031 0.05664063 0.3359375 ]
 [0.30957031 1.         0.0234375  0.08886719]
 [0.05664063 0.0234375  1.         0.26367188]
 [0.3359375  0.08886719 0.26367188 1.        ]]


In [ ]:
# CASE 4, standard implementation on a QPU

fidelity_QPU = ComputeUncompute(sampler=sampler_QPU,  pass_manager=pass_manager_QPU)
qkernel_QPU = FidelityQuantumKernel(feature_map = feature_map, fidelity = fidelity_QPU)
kernel_matrix_QPU = qkernel_QPU.evaluate(X)

print(kernel_matrix_QPU)

#NOTE: this run used 8 seconds and 6 jobs on  ibm_sherbrooke

[[1.         0.29199219 0.03295898 0.29370117]
 [0.29199219 1.         0.08935547 0.09057617]
 [0.03295898 0.08935547 1.         0.24731445]
 [0.29370117 0.09057617 0.24731445 1.        ]]
